In [1]:
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, Accuracy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam,schedules
from sklearn.metrics import confusion_matrix
import numpy as np




In [2]:
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import random
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
TRAIN_SPLIT = 0.85

DIR = './'
imgH = 32
imgW = 32

# file = open('data_index.txt', 'w')
count = 0

def load_image_data():
    imgDir = DIR + 'Normal/'
    imgSet1 = prepare_image_array(imgDir, imgW, imgH)
    m = imgSet1.shape[0]

    imgDir = DIR + 'OverExposed/'
    imgSet2 = prepare_image_array(imgDir, imgW, imgH)
    n = imgSet2.shape[0]

    imgDir = DIR + 'UnderExposed/'
    imgSet3 = prepare_image_array(imgDir, imgW, imgH)
    o = imgSet3.shape[0]

    # Put all image data into one array.
    imgSet = np.concatenate((imgSet1, imgSet2, imgSet3), axis=0)
    print(imgSet.shape)

    labelSet1 = np.zeros(m, dtype=np.uint8)
    labelSet2 = np.ones(n, dtype=np.uint8)
    label_o = np.ones(o, dtype=np.uint8)
    labelSet3 = np.add(label_o, label_o)
    labelSet = np.concatenate((labelSet1, labelSet2, labelSet3), axis=0)


    p = imgSet.shape[0]  # p = n + m + o

    # random.shuffle(indices)
    # for i in range(len(indices)):
    #     file.write(str(indices[i])+",")
    
    # file.close()
    f = open('data_index.txt', 'r')
    indices = []
    for x in f.read().split(','):
        if len(x) > 0:
            num = ''
            for j in x:
                if(j>='0' and j<='9'):
                    num +=j;
            if len(num)>0 and  int(num)<p:
                indices.append(int(num)) 
   
    # print(indices)
    imgSet = imgSet[indices]
    labelSet = labelSet[indices]
    print(labelSet[:5])
    print("Label set : ")
    labelSet = labelSet.reshape(p, 1)

#   one hot encoding

    labelSet = to_categorical(labelSet)

    # print(labelSet[:5])
    
    # print(imgSet[0])
    imgSet = imgSet.astype(np.float32)
    imgSet = imgSet/255.0

    r = int(p * TRAIN_SPLIT)
    trainX = imgSet[:r]
    trainY = labelSet[:r]
    testX = imgSet[r:]
    testY = labelSet[r:]

    return trainX, trainY, testX, testY



def prepare_image_array(imgDir, imgW, imgH):
    imgList = os.listdir(imgDir)
    # print(imgList)
    n = len(imgList)

    imgSet = []
    for i in range(n):
        imgPath = imgDir + imgList[i]
        if (os.path.exists(imgPath)):
            # print(imgPath)
            img = cv2.imread(imgPath)
            resizedImg = cv2.resize(img, (imgW, imgH))
            rgbImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
            imgSet.append(rgbImg)
        else:
            print("It is not a valid image path.")

    # print("total image "+str(len(imgSet)))
    imgSet = np.array(imgSet, dtype=np.uint8)
    # print("total shape "+str(imgSet.shape))

    return imgSet



In [3]:
train_x,train_Y, testInSet, testOutSet =load_image_data()


(8984, 32, 32, 3)
[0 2 1 2 0]
Label set : 


In [4]:
from sklearn.metrics import classification_report

def run_model(modelpath,testInSet, testOutSet):

    lr_schedule = schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=10000,
        decay_rate=0.9)

    sgd = SGD(learning_rate=lr_schedule)


    model = load_model(modelpath, compile = True)
    model.compile(loss = 'mse', optimizer = sgd, metrics = ['accuracy', Recall(), Precision()])
    


#     y_test = np.argwhere(testOutSet)[:,1]
    # print(y_test)
#     indices = np.argwhere(y_test == 0).flatten()
#     print(indices.shape)
#     testInNormal = testInSet[indices]
#     testOutNormal = testOutSet[indices]



#     indices = np.argwhere(y_test == 1).flatten()
#     testInOver = testInSet[indices]
#     testOutOver = testOutSet[indices]
#     indices = np.argwhere(y_test == 2).flatten()
#     testInUnder = testInSet[indices]
#     testOutUnder = testOutSet[indices]



    '''	Test Model. '''
    lossT, accT, precisionT, recallT = model.evaluate(testInSet, testOutSet)
#     lossN, accN, precisionN, recallN = model.evaluate(testInNormal, testOutNormal)
#     lossO, accO, precisionO, recallO = model.evaluate(testInOver, testOutOver)
#     lossU, accU, precisionU, recallU = model.evaluate(testInUnder, testOutUnder)


    print("\n\nlossT = %s, accT = %s,  precisionT = %s, recallT = %s "%(lossT, accT, precisionT, recallT))
#     print("lossN = %s, accN = %s,  precisionN = %s, recallN = %s "%(lossN, accN, precisionN, recallN))
#     print("lossO = %s, accO = %s,  precisionO = %s, recallO = %s "%(lossO, accO, precisionO, recallO))
#     print("lossU = %s, accU = %s,  precisionU = %s, recallU = %s "%(lossU, accU, precisionU, recallU))

    predicted_output = []
    prediction = model.predict(testInSet)
    for i in prediction:
        predicted_output.append(np.argmax(i)) 

    actual_output = []
    for i in testOutSet:
        actual_output.append(np.argmax(i)) 
    con_mat = confusion_matrix(actual_output, predicted_output)
#     for i in con_mat:
#         print(i)
        
    cm =  classification_report(actual_output,predicted_output )
#     print(cm)
    cm = cm.split(" ")
    count = 0
    cnt = 0
    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "
    acc = "Accuracy & \multicolumn{3}{r}{"
    mac = "Macro Avg "
    weight = "Weighted Avg "
    for i in cm:
        if len(i)==4:
            count = 0
            for j in i:
                if j=='.':
                    count =1
            if count ==1:
                if(cnt<3):
                    first_line +=" & %s"%i[2:]
                    first_line +='\%'
                if cnt>=3 and cnt<6:
                    second_line += " & %s"%i[2:]
                    second_line +='\%'
                if cnt>=6 and  cnt<9:
                    third_line += " & %s"%i[2:]
                    third_line +='\%'
                if cnt>=9 and cnt<10:
                    acc += " %s"%i[2:]
                    acc +='\%}'
                if cnt>=10 and cnt<13:
                    mac +=" & %s"%i[2:]
                    mac +='\%'
                if cnt>=13 and cnt<16:
                    weight += " & %s"%i[2:]
                    weight +='\%'
                cnt+=1

    print(first_line+'\\\\')
    print(second_line+'\\\\')
    print(third_line+'\\\\\\hline')
    print(acc+'\\\\')
    print(mac+'\\\\')
    print(weight+'\\\\\hline\hline\\thinlines')



    print("\n\n\n\n")


    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "

    count = 0
    for i in con_mat:
        if count==0:
            first_line+= ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==1:
            second_line+=  ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==2:
            third_line+=   ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        count+=1


    print(first_line+" \\\\\hline")
    print(second_line+" \\\\\hline")
    print(third_line+" \\\\\hline")

    
    return cm,con_mat

    

In [5]:
print("Non trainable")
DIR = './non-trainable/'
modelpath = DIR + 'VGG_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Non trainable
43/43 [==============================] - 3s 68ms/step - loss: 0.2204 - accuracy: 0.4221 - recall: 0.0000e+00 - precision: 0.0000e+00


lossT = 0.22044524550437927, accT = 0.4221068322658539,  precisionT = 0.0, recallT = 0.0 
Normal  & 34\% & 59\% & 44\%\\
Over Expsoed  & 42\% & 34\% & 38\%\\
Under Expsoed  & 63\% & 35\% & 45\%\\\hline
Accuracy & \multicolumn{3}{r}{ 42\%}\\
Macro Avg  & 47\% & 43\% & 42\%\\
Weighted Avg  & 47\% & 42\% & 42\%\\\hline\hline\thinlines





Normal  & 248 & 112 & 61 \\\hline
Over Expsoed  & 262 & 155 & 35 \\\hline
Under Expsoed  & 209 & 100 & 166 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.34',
  '',
  '',
  '',
  '',
  '',
  '0.59',
  '',
  '',
  '',
  '',
  '',
  '0.44',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.42',
  '',
  '',
  '',
  '',
  '',
  '0.34',
  '',
  '',
  '',
  '',
  '',
  '0.38',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.63',
  '',
  '',
  '',
  '',
  '',
  '0.35',
  '',
  '',
  '',
  '',
  '',
  '0.45',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [6]:
print("trainable")
DIR = './trainable/'
modelpath = DIR + 'trainable-tf-Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

trainable
43/43 [==============================] - 3s 67ms/step - loss: 0.1599 - accuracy: 0.6521 - recall_1: 0.4726 - precision_1: 0.7547


lossT = 0.15991602838039398, accT = 0.6520771384239197,  precisionT = 0.47255194187164307, recallT = 0.7547393441200256 
Normal  & 50\% & 27\% & 35\%\\
Over Expsoed  & 68\% & 78\% & 72\%\\
Under Expsoed  & 69\% & 87\% & 77\%\\\hline
Accuracy & \multicolumn{3}{r}{ 65\%}\\
Macro Avg  & 62\% & 64\% & 61\%\\
Weighted Avg  & 63\% & 65\% & 62\%\\\hline\hline\thinlines





Normal  & 113 & 161 & 147 \\\hline
Over Expsoed  & 59 & 352 & 41 \\\hline
Under Expsoed  & 53 & 8 & 414 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.50',
  '',
  '',
  '',
  '',
  '',
  '0.27',
  '',
  '',
  '',
  '',
  '',
  '0.35',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.68',
  '',
  '',
  '',
  '',
  '',
  '0.78',
  '',
  '',
  '',
  '',
  '',
  '0.72',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.69',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '0.77',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [7]:
print("fully connected ")
DIR = './Fully-Connnected-model/'
modelpath = DIR + 'Fully_connected_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

fully connected 
43/43 [==============================] - 1s 4ms/step - loss: 0.1041 - accuracy: 0.7878 - recall_2: 0.7849 - precision_2: 0.7884


lossT = 0.10410906374454498, accT = 0.7878338098526001,  precisionT = 0.784866452217102, recallT = 0.7883755564689636 
Normal  & 70\% & 64\% & 67\%\\
Over Expsoed  & 87\% & 78\% & 82\%\\
Under Expsoed  & 79\% & 93\% & 85\%\\\hline
Accuracy & \multicolumn{3}{r}{ 79\%}\\
Macro Avg  & 79\% & 78\% & 78\%\\
Weighted Avg  & 79\% & 79\% & 79\%\\\hline\hline\thinlines





Normal  & 271 & 51 & 99 \\\hline
Over Expsoed  & 83 & 351 & 18 \\\hline
Under Expsoed  & 34 & 1 & 440 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.70',
  '',
  '',
  '',
  '',
  '',
  '0.64',
  '',
  '',
  '',
  '',
  '',
  '0.67',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '0.78',
  '',
  '',
  '',
  '',
  '',
  '0.82',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.79',
  '',
  '',
  '',
  '',
  '',
  '0.93',
  '',
  '',
  '',
  '',
  '',
  '0.85',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [8]:
print("smalll")
DIR = './Small-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Small_CNN_Classifier.hdf5'

cm,con_mat = run_model(modelpath,testInSet, testOutSet)




smalll
43/43 [==============================] - 0s 4ms/step - loss: 0.1068 - accuracy: 0.7774 - recall_3: 0.7678 - precision_3: 0.7841


lossT = 0.1067890003323555, accT = 0.7774480581283569,  precisionT = 0.7678041458129883, recallT = 0.7840909361839294 
Normal  & 67\% & 65\% & 66\%\\
Over Expsoed  & 86\% & 76\% & 81\%\\
Under Expsoed  & 79\% & 91\% & 85\%\\\hline
Accuracy & \multicolumn{3}{r}{ 78\%}\\
Macro Avg  & 78\% & 77\% & 77\%\\
Weighted Avg  & 78\% & 78\% & 78\%\\\hline\hline\thinlines





Normal  & 272 & 54 & 95 \\\hline
Over Expsoed  & 89 & 344 & 19 \\\hline
Under Expsoed  & 42 & 1 & 432 \\\hline


In [9]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Large-Size-Convolution-Maxpooling-model
43/43 [==============================] - 2s 29ms/step - loss: 0.1230 - accuracy: 0.7300 - recall_4: 0.7188 - precision_4: 0.7352


lossT = 0.12298060208559036, accT = 0.7299703359603882,  precisionT = 0.7188427448272705, recallT = 0.7352048754692078 
Normal  & 63\% & 48\% & 54\%\\
Over Expsoed  & 88\% & 73\% & 80\%\\
Under Expsoed  & 69\% & 96\% & 80\%\\\hline
Accuracy & \multicolumn{3}{r}{ 73\%}\\
Macro Avg  & 73\% & 72\% & 71\%\\
Weighted Avg  & 74\% & 73\% & 72\%\\\hline\hline\thinlines





Normal  & 200 & 44 & 177 \\\hline
Over Expsoed  & 99 & 328 & 25 \\\hline
Under Expsoed  & 18 & 1 & 456 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.63',
  '',
  '',
  '',
  '',
  '',
  '0.48',
  '',
  '',
  '',
  '',
  '',
  '0.54',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.88',
  '',
  '',
  '',
  '',
  '',
  '0.73',
  '',
  '',
  '',
  '',
  '',
  '0.80',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.69',
  '',
  '',
  '',
  '',
  '',
  '0.96',
  '',
  '',
  '',
  '',
  '',
  '0.80',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

###### 

In [ ]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)a  = {}